# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [67]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [68]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,1.85,93,100,1.54,CA,1686216492
1,1,hermanus,-34.4187,19.2345,16.38,81,99,10.16,ZA,1686216493
2,2,lebu,-37.6167,-73.6500,14.44,86,99,11.87,CL,1686216495
3,3,isetskoye,56.4858,65.3226,19.98,45,68,3.84,RU,1686216495
4,4,vilyuchinsk,52.9306,158.4028,14.15,77,50,1.50,RU,1686216496


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [69]:
%%capture --no-display

# Configure the map plot
holiday_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map
holiday_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [70]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_conditions = city_data_df.loc[(city_data_df["Max Temp"] <= 25) & (city_data_df["Max Temp"] >= 20)&
                                      (city_data_df["Wind Speed"] <= 3.5) & 
                                      (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
perfect_conditions_df = perfect_conditions.reset_index(drop=True)

# Display sample data
perfect_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,36,karratha,-20.7377,116.8463,24.86,33,0,3.25,AU,1686216515
1,78,port elizabeth,-33.9180,25.5701,22.95,51,0,2.57,ZA,1686216539
2,133,hermosillo,29.0667,-110.9667,23.80,49,0,0.45,MX,1686216467
3,138,lake charles,30.2131,-93.2044,22.00,94,0,2.06,US,1686216575
4,214,east london,-33.0153,27.9116,21.52,43,0,2.06,ZA,1686216436
5,263,dyatlovo,53.4631,25.4068,22.72,52,0,1.34,BY,1686216651
6,308,sao vicente,-23.9631,-46.3919,20.15,26,0,1.76,BR,1686216677
7,336,grosshansdorf,53.6667,10.2833,21.03,66,0,2.57,DE,1686216717
8,433,vostochnyy,55.8167,37.8667,21.99,38,0,1.76,RU,1686216800
9,474,volkovysk,53.1561,24.4513,21.82,55,0,2.49,BY,1686216824


### Step 3: Create a new DataFrame called `hotel_df`.

In [71]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_conditions_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,karratha,AU,-20.7377,116.8463,33,
1,port elizabeth,ZA,-33.9180,25.5701,51,
2,hermosillo,MX,29.0667,-110.9667,49,
3,lake charles,US,30.2131,-93.2044,94,
4,east london,ZA,-33.0153,27.9116,43,
5,dyatlovo,BY,53.4631,25.4068,52,
6,sao vicente,BR,-23.9631,-46.3919,26,
7,grosshansdorf,DE,53.6667,10.2833,66,
8,vostochnyy,RU,55.8167,37.8667,38,
9,volkovysk,BY,53.1561,24.4513,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [72]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": 'accommodation.hotel',
          "limit": 1,
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
karratha - nearest hotel: Karratha International Hotel
port elizabeth - nearest hotel: Waterford Hotel
hermosillo - nearest hotel: Lucerna
lake charles - nearest hotel: Howard Johnson by Wyndham Historic Lake Charles
east london - nearest hotel: No hotel found
dyatlovo - nearest hotel: Дзядзька Ваня
sao vicente - nearest hotel: Pousada Vitória
grosshansdorf - nearest hotel: Ringhotel Ahrensburg
vostochnyy - nearest hotel: Оранж
volkovysk - nearest hotel: Ваяж
mahina - nearest hotel: Motu Martin
seminole - nearest hotel: Best Western Plus Sanford


,City,Country,Lat,Lng,Humidity,Hotel Name
0,karratha,AU,-20.7377,116.8463,33,Karratha International Hotel
1,port elizabeth,ZA,-33.9180,25.5701,51,Waterford Hotel
2,hermosillo,MX,29.0667,-110.9667,49,Lucerna
3,lake charles,US,30.2131,-93.2044,94,Howard Johnson by Wyndham Historic Lake Charles
4,east london,ZA,-33.0153,27.9116,43,No hotel found
5,dyatlovo,BY,53.4631,25.4068,52,Дзядзька Ваня
6,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
7,grosshansdorf,DE,53.6667,10.2833,66,Ringhotel Ahrensburg
8,vostochnyy,RU,55.8167,37.8667,38,Оранж
9,volkovysk,BY,53.1561,24.4513,55,Ваяж


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [73]:
%%capture --no-display

# Configure the map plot
hotel_locations = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_locations 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)